<a href="https://colab.research.google.com/github/khodozzz/fuzzy-logic-task/blob/main/fuzzy_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings

warnings.filterwarnings('ignore')

### *Кусочно-линейные фунции*

In [ ]:
def pi_function(x, a, b, c, d):
    if a <= x <= b:
        return (x - a) / (b - a)
    elif b < x <= c:
        return 1
    elif c < x <= d:
        return (d - x) / (d - c)
    else:
        return 0


def l_function(x, a, b, c):
    if a <= x <= b:
        return (x - a) / (b - a)
    elif b < x <= c:
        return (c - x) / (c - b)
    else:
        return 0


def z_function(x, a, b):
    if x <= a:
        return 1
    if a < x <= b:
        return (b - x) / (b - a)
    else:
        return 0


def s_function(x, a, b):
    if x <= a:
        return 0
    if a < x <= b:
        return (x - a) / (b - a)
    else:
        return 1

# Формирование базы нечетки правил

## Формирование нечетких переменных

In [ ]:
class LinguisticVariable:
    def __init__(self, term_functions, borders):
        self.term_functions = term_functions
        self.borders = borders
        self.space = np.linspace(self.borders[0], self.borders[1], 100)

    def fazzification(self, value):
        return {term: self.term_functions[term](value) for term in self.term_functions}

    def defazzification(self, composition, method='left max'):
        if method == 'left max':
            idx = np.argmax(composition)
            x = self.space[idx]
        elif method == 'right max':
            tmp = composition[::-1]
            idx = len(tmp) - np.argmax(tmp) - 1
            x = self.space[idx]
        elif method == 'center of gravity':
            x = np.average(self.space, axis=0, weights=composition)
        else:
            raise Exception("no such method")

        return x

    def plot_activation(self, functions):
        fig = go.Figure()
        for idx, func in enumerate(functions):
            fig.add_trace(go.Scatter(x=self.space, y=[func(x) for x in self.space],
                                     mode='lines', name=f'Правило {idx}'))
        fig.show()

    def plot_composition(self, composition):
        fig = go.Figure(data=go.Scatter(x=self.space, y=composition))
        fig.show()

    def plot_terms(self):
        fig = go.Figure()
        for term in self.term_functions:
            fig.add_trace(go.Scatter(x=self.space, y=[self.term_functions[term](x) for x in self.space],
                                     mode='lines', name=term))
        fig.show()

### Входные переменные

In [ ]:
"""
• Профессиональные знания.
    Множество определения = [0, 16].
    Множество термов = {высокий уровень, выше среднего, ниже среднего, слабые знания}.
"""

terms = {'слабые знания': lambda x: z_function(x, 0, 5),
         'ниже среднего': lambda x: pi_function(x, 0, 5, 9, 10),
         'выше среднего': lambda x: pi_function(x, 9, 10, 12, 13),
         'высокий уровень': lambda x: s_function(x, 12, 13)}

knowledge_var = LinguisticVariable(terms, (0, 16))

knowledge_var.plot_terms()

In [ ]:
"""
• Коммуникабельность.
    Множество определения = [0, 10].
    Множество термов = {практически отсутствует, вполне в норме, высокая}
"""

terms = {'практически отсутствует': lambda x: z_function(x, 3, 4),
         'вполне в норме': lambda x: pi_function(x, 3, 4, 6, 7),
         'высокая': lambda x: s_function(x, 6, 7)}

communicability_var = LinguisticVariable(terms, (0, 10))

communicability_var.plot_terms()

In [ ]:
"""
• Внешний вид.
    Множество определения = [0, 8].
    Множество термов = {плохой, приличный, отличный}.
"""

terms = {'плохой': lambda x: z_function(x, 1, 3),
         'приличный': lambda x: pi_function(x, 1, 3, 5, 7),
         'отличный': lambda x: s_function(x, 5, 7)}

appereance_var = LinguisticVariable(terms, (0, 8))

appereance_var.plot_terms()

In [ ]:
"""
• Деловые качества.
    Множество определения = [0, 14].
    Множество термов = {слабые, не очень хорошие, достаточные, выдающиеся}.
"""

terms = {'слабые': lambda x: z_function(x, 1, 5),
         'не очень хорошие': lambda x: pi_function(x, 1, 5, 9, 11),
         'достаточные': lambda x: pi_function(x, 9, 11, 12, 14),
         'выдающиеся': lambda x: s_function(x, 12, 14)}

business_qualities_var = LinguisticVariable(terms, (0, 14))

business_qualities_var.plot_terms()

### Выходные переменные

In [ ]:
"""
• Прохождение отбора.
    Множество определения = [0, 8].
    Множество термов = {лучше не брать, можно взять, ценный кадр}.
"""

terms = {'лучше не брать': lambda x: z_function(x, 2, 3),
         'можно взять': lambda x: pi_function(x, 2, 3, 5, 7),
         'ценный кадр': lambda x: s_function(x, 5, 7)}

selection_choise_var = LinguisticVariable(terms, (0, 8))

selection_choise_var.plot_terms()

## Формирование правил

In [ ]:
import pandas as pd


class Rules(pd.DataFrame):
    input_columns = None
    output_column = None
    legitimacy_column = None

    def set_input_columns(self, columns):
        self.input_columns = columns

    def set_output_column(self, column):
        self.output_column = column

    def set_legitimacy_column(self, column):
        self.legitimacy_column = column

    def calc_values(self, values_list):
        self['Value'] = [min([value[rule]
                              for value, rule in zip(values_list, row[self.input_columns])]) *
                         row[self.legitimacy_column] for idx, row in self.iterrows()]
        

In [ ]:
input_columns = ['Профессиональные знания',
                 'Коммуникабельность',
                 'Внешний вид',
                 'Деловые качества']
output_column = 'Прохождение отбора'
legitimacy_column = 'Степень достоверности правила'

rules = Rules(columns=input_columns + [output_column] + [legitimacy_column])

rules.set_input_columns(input_columns)
rules.set_output_column(output_column)
rules.set_legitimacy_column(legitimacy_column)

rules.loc[len(rules)] = ['высокий уровень', 'высокая', 'плохой', 'слабые', 'можно взять', 0.8]
rules.loc[len(rules)] = ['высокий уровень', 'практически отсутствует', 'отличный', 'не очень хорошие', 'можно взять',
                         0.7]
rules.loc[len(rules)] = ['высокий уровень', 'вполне в норме', 'приличный', 'достаточные', 'ценный кадр', 0.9]
rules.loc[len(rules)] = ['высокий уровень', 'высокая', 'плохой', 'выдающиеся', 'ценный кадр', 0.9]

rules.loc[len(rules)] = ['выше среднего', 'вполне в норме', 'приличный', 'слабые', 'можно взять', 0.6]
rules.loc[len(rules)] = ['выше среднего', 'вполне в норме', 'отличный', 'не очень хорошие', 'ценный кадр', 0.7]
rules.loc[len(rules)] = ['выше среднего', 'вполне в норме', 'приличный', 'достаточные', 'можно взять', 1.0]
rules.loc[len(rules)] = ['выше среднего', 'высокая', 'приличный', 'выдающиеся', 'ценный кадр', 1.0]

rules.loc[len(rules)] = ['ниже среднего', 'практически отсутствует', 'плохой', 'слабые', 'лучше не брать', 0.7]
rules.loc[len(rules)] = ['ниже среднего', 'вполне в норме', 'приличный', 'не очень хорошие', 'можно взять', 0.4]
rules.loc[len(rules)] = ['ниже среднего', 'вполне в норме', 'плохой', 'достаточные', 'можно взять', 0.8]
rules.loc[len(rules)] = ['ниже среднего', 'практически отсутствует', 'отличный', 'выдающиеся', 'можно взять', 0.8]

rules.loc[len(rules)] = ['слабые знания', 'вполне в норме', 'плохой', 'слабые', 'лучше не брать', 1.0]
rules.loc[len(rules)] = ['слабые знания', 'вполне в норме', 'приличный', 'не очень хорошие', 'лучше не брать', 0.9]
rules.loc[len(rules)] = ['слабые знания', 'практически отсутствует', 'плохой', 'достаточные', 'лучше не брать', 0.3]
rules.loc[len(rules)] = ['слабые знания', 'высокая', 'приличный', 'выдающиеся', 'можно взять', 0.4]

rules

,Профессиональные знания,Коммуникабельность,Внешний вид,Деловые качества,Прохождение отбора,Степень достоверности правила
0,высокий уровень,высокая,плохой,слабые,можно взять,0.8
1,высокий уровень,практически отсутствует,отличный,не очень хорошие,можно взять,0.7
2,высокий уровень,вполне в норме,приличный,достаточные,ценный кадр,0.9
3,высокий уровень,высокая,плохой,выдающиеся,ценный кадр,0.9
4,выше среднего,вполне в норме,приличный,слабые,можно взять,0.6
5,выше среднего,вполне в норме,отличный,не очень хорошие,ценный кадр,0.7
6,выше среднего,вполне в норме,приличный,достаточные,можно взять,1.0
7,выше среднего,высокая,приличный,выдающиеся,ценный кадр,1.0
8,ниже среднего,практически отсутствует,плохой,слабые,лучше не брать,0.7
9,ниже среднего,вполне в норме,приличный,не очень хорошие,можно взять,0.4


# *Утилиты*

In [ ]:
class Activator:
    def __init__(self, rules):
        self.rules = rules
        
    @staticmethod
    def rule_activation(output_rule, rule_value, var, method='min'):
        m = var.term_functions[output_rule]
        if method == 'min':
            return lambda x: min(rule_value, m(x))
        elif method == 'prod':
            return lambda x: rule_value * m(x)
        elif method == 'average':
            return lambda x: (rule_value + m(x)) / 2
        else:
            raise Exception("no such method")   
                
    def activation_functions(self, output_var, method='min'):
        return [self.rule_activation(row[self.rules.output_column], rule_value, output_var, method)
                for rule_value, (idx, row) in zip(self.rules.Value, self.rules.iterrows())]


In [ ]:
class Composer:
    def composition(self, functions, var, method='max'):
        if method == 'max':
            return [max([func(x) for func in functions]) for x in var.space]
        elif method == 'sum':
            return [sum([func(x) for func in functions]) for x in var.space]
        else:
            raise Exception("no such method")

# Использование системы

In [ ]:
#@title Входные данные

knowledge_mark =  9.7#@param {type:"number"}
communicability_mark =  6.5#@param {type:"number"}
appereance_mark = 6.5 #@param {type:"number"} 
business_qualities_mark = 12.5 #@param {type:"number"}

## Фаззификация

In [ ]:
knowledge_values = knowledge_var.fazzification(knowledge_mark)
communicability_values = communicability_var.fazzification(communicability_mark)
appereance_values = appereance_var.fazzification(appereance_mark)
business_qualities_values = business_qualities_var.fazzification(business_qualities_mark)

print('Профессиональные знания:', knowledge_values)
print('Коммуникабельность:', communicability_values)
print('Внешний вид:',appereance_values)
print('Деловые качества:', business_qualities_values)

Профессиональные знания: {'слабые знания': 0, 'ниже среднего': 0.3000000000000007, 'выше среднего': 0.6999999999999993, 'высокий уровень': 0}
Коммуникабельность: {'практически отсутствует': 0, 'вполне в норме': 0.5, 'высокая': 0.5}
Внешний вид: {'плохой': 0, 'приличный': 0.25, 'отличный': 0.75}
Деловые качества: {'слабые': 0, 'не очень хорошие': 0, 'достаточные': 0.75, 'выдающиеся': 0.25}


## Агрегация

In [ ]:
rules.calc_values([knowledge_values, communicability_values, appereance_values, business_qualities_values])
rules[rules.Value != 0]

,Профессиональные знания,Коммуникабельность,Внешний вид,Деловые качества,Прохождение отбора,Степень достоверности правила,Value
6,выше среднего,вполне в норме,приличный,достаточные,можно взять,1.0,0.25
7,выше среднего,высокая,приличный,выдающиеся,ценный кадр,1.0,0.25


## Активация

In [ ]:
#@title Метод активации { form-width: "550px" }
method = "min" #@param ["min", "prod", "average"]

activator = Activator(rules)
activation_functions = activator.activation_functions(selection_choise_var, 
                                                      method=method)

selection_choise_var.plot_activation(activation_functions)

## Композиция

In [ ]:
#@title Метод композиции { form-width: "550px" }
method = "max" #@param ["max", "sum"]

composer = Composer()
composition = composer.composition(activation_functions, 
                                   selection_choise_var, method=method)

selection_choise_var.plot_composition(composition)

## Дефаззификация

In [ ]:
#@title Метод дефаззификации { form-width: "550px" }
method = "center of gravity" #@param ['left max', 'right max', 'center of gravity']

defaz = selection_choise_var.defazzification(composition, 
                                             method=method)
print(defaz)
print(selection_choise_var.fazzification(defaz))

5.189329944248574
{'лучше не брать': 0, 'можно взять': 0.9053350278757128, 'ценный кадр': 0.09466497212428715}
